# Lecture 2 - Introduction to SparkSQL  in pyspark

First, start your spark kernel in a jupyter notebook. This will differ depending on whether you are on your local laptop, or the CS resources. Google for setting up your Jupyter notebook with spark - it is a matter of setting a few options. We will use both local laptop and CS resources interchangably. 

Executing any command in a new notebook with start the spark session. It used to be in Spark 1.x that there were two main classes called SparkContext and SQLContext, dealing with the session-related stuff. You could only create one per application. Later on, it was found to be limitting, so a new class was introduced - SparkSession. 

In [1]:
2+2

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
110,application_1536701919660_0039,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
4

In [5]:
dir(spark)  # get methods for spark
sc = spark.sparkContext
print(spark.version)

2.2.0.2.6.3.0-235

In [4]:
dir(sc)

['PACKAGE_EXTENSIONS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accumulatorServer', '_active_spark_context', '_batchSize', '_callsite', '_checkpointFile', '_conf', '_dictToJavaMap', '_do_init', '_ensure_initialized', '_gateway', '_getJavaStorageLevel', '_initialize_context', '_javaAccumulator', '_jsc', '_jvm', '_lock', '_next_accum_id', '_pickled_broadcast_vars', '_python_includes', '_repr_html_', '_temp_dir', '_unbatched_serializer', 'accumulator', 'addFile', 'addPyFile', 'appName', 'applicationId', 'binaryFiles', 'binaryRecords', 'broadcast', 'cancelAllJobs', 'cancelJobGroup', 'defaultMinPartitions', 'defaultParallelism', 'dump_profiles', 'em

## RDD's

In [ ]:
rdd = sc.parallelize([1,2,3,4])   # create RDD from a list
rdd.collect()
mylist = range(0, 10**8)          # a lot of elements
rddlarge = sc.parallelize(mylist)

In [ ]:
from operator import add

import time

## elapsed time 
st = time.clock()   #time in seconds
rddlarge.map(lambda x: x*x).reduce(add)
et = time.clock()
print( "elapsed time is {} seconds".format(et - st))  # elapsed time

from functools import reduce
st = time.clock()
reduce( add, map(lambda x: x*x, mylist))
et = time.clock()
print( "elapsed time is {} seconds".format(et - st))  # elapsed time


In [ ]:
### filter examples: 
rddlarge.filter( lambda x: x % 1000 == 0).take(10)
filter(lambda x: x % 100000 == 0, mylist)  #not what you expect... 


In [ ]:
############### text file - transformations and actions
rddtext = sc.textFile("/home/kris/datasets/text/jane_austen.txt")

rddwords = rddtext.map(lambda ll: ll.split(" "))
for i in rddwords.take(10):
   print(i)

rddwords2 = rddwords.flatMap(lambda x: x)
for i in rddwords2.take(10):
   print(i)



# DataFrames

In [ ]:
############### make a dataframe
bb = spark.createDataFrame(data=[(1,2, 'this', 10.0), (3,4, 'Is', 3.14)], schema=("A", "B", "word", "somedouble"))  ## from some 
bb.withColumn("E", bb.A + bb.B).show()
bb.filter(bb.word == "Is" ).show()
bb[bb.word.startswith("I")]

from pyspark.sql.functions import col
bb.filter(col('word').startswith("I")).show()
bb.filter(bb.word.startswith("I")).show()


In [ ]:
############### to read a dataframe
localfilename = "/home/kris/datasets/small/iris.csv"
filename = "/home/kp807/iris.csv"
hdfsfilename = "/user/kp807/smalldata/iris.csv"

df = spark.read.option("header", "true").option("inferSchema", "true").csv(filename)
df.cache()
df.count()
# now look at the Storage tab in the Spark UI



In [ ]:
df.show()

df.printSchema()

In [15]:
df.registerTempTable("mydf")
df1 = sqlContext.sql("SELECT SUM(sepal_length) FROM mydf")
#sql("SELECT SUM(sepal_length) AS sum_sep_len FROM mydf").show()
#df.schema
#df.printSchema()
############### to use a dataframe in SQL statements: 
df.registerTempTable("iris")
spark.sql("select avg(petal_length) AS avg_petal_length from iris").show()


In [25]:
df.registerTempTable("iris")
spark.sql("SELECT species, count(1) from iris GROUP BY species").show()

+---------------+--------+
|        species|count(1)|
+---------------+--------+
| Iris-virginica|      50|
|    Iris-setosa|      50|
|Iris-versicolor|      50|
+---------------+--------+

In [29]:
df.createOrReplaceTempView("iris")
sqlDF = spark.sql("SELECT * FROM iris")
sqlDF.show(3, False)


+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|species    |
+------------+-----------+------------+-----------+-----------+
|5.1         |3.5        |1.4         |0.2        |Iris-setosa|
|4.9         |3.0        |1.4         |0.2        |Iris-setosa|
|4.7         |3.2        |1.3         |0.2        |Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 3 rows

# UDF functions

In [ ]:
##############  UDF functions
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
spsuff = udf(lambda s: s.split("-")[1], StringType())   #Iris-setosa --> setosa
df.select(spsuff("species").alias("species2")).show()
df.withColumn(spsuff("species").alias("species2")).show()

sqlContext.registerFunction("species_suffix", slen)
df.selectExpr("*", "species_suffix(species AS species2")).show()

                                                           

# SQL analogues

In [ ]:
### entire group by statement: 
df.select("petal_length", "species").filter("petal_length>3").groupBy("species").count().sort("count", ascending=False).show()
df.filter("petal_length>3").groupBy("species").mean().show()


# Joins

In [ ]:
valuesA = [('Alice',25),('Bob',32),('Charlie',34),('Dave',24)]
dfage = spark.createDataFrame(valuesA,['name','age']).cache()
valuesB = [('Alice',"Research"),('Bob',"Sales"),('Charlie',"Sales"),('Evan',"Marketing")]
dfdept = spark.createDataFrame(valuesB,['name','dept']).cache()

dfage.join(dfdept).show()
dfage.join(dfdept, "name").show()
dfage.join(dfdept, "name", how = "left").show()
dfage.join(dfdept, "name", how = "right").show()
dfage.join(dfdept, "name", how = "outer").show()

dfage.withColumnRenamed("name", "name2").join(dfdept, col("name") == col("name2")).show()



# Other formats

In [8]:
df.write.json(DIR+"iris.json")
dftemp = spark.read.json(DIR+"iris.json")
dftemp.show()

name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [ ]:
df.write.parquet(DIR+"iris.parquet")
dftemp = spark.read.parquet(DIR+"iris.parquet")
dftemp.show()


# Some bugs

In [6]:
############### some bugs

rdd.toDF().show()    #fails 
rdd.zipWithIndex().toDF().show()


name 'rdd' is not defined
Traceback (most recent call last):
NameError: name 'rdd' is not defined



# Practice time!

In [7]:
filename = "churn.csv"
df2 = spark.read.option("header", "true").option("inferSchema", "true").csv(filename)
df2.cache()
df2.count()


'Path does not exist: hdfs://data-services2.cs.rutgers.edu:8020/user/kp807/filename;'
Traceback (most recent call last):
  File "/hadoop/yarn/local/usercache/kp807/appcache/application_1536701919660_0039/container_e49_1536701919660_0039_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 410, in csv
    return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/hadoop/yarn/local/usercache/kp807/appcache/application_1536701919660_0039/container_e49_1536701919660_0039_01_000001/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hadoop/yarn/local/usercache/kp807/appcache/application_1536701919660_0039/container_e49_1536701919660_0039_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: hdfs://data-services2.cs.rutgers.edu:8020/user/kp807/filename

## Your turn to experiment

- save as parquet - and inspect the files
- try repartitioning and observe what happens
- save as table
- find a way to do a Cartesian product (usually that's an antipattern)
- parse dates - read on DateTypes
- make a new column with dates 30 days back
- experiment with 5 new column functions 
- make a list of all differences that you see between pandas and pyspark dataframes